In [ ]:
%%HTML
<style>
    body {
        --vscode-font-family: "Noto Serif"
    }
</style>

# Segment Tree vs Fenwick Tree (Binary Indexed Tree)

Both are **data structures** designed for efficient **range queries and updates** on arrays.

---

## 1. Segment Tree

- A **binary tree** structure that stores information about intervals (segments) of an array.  
- Each node represents a segment `[L, R]` and stores an aggregate (e.g., sum, min, max).  
- Root covers the entire array, children cover halves.  

**Operations:**
- Query (e.g., sum of [l, r]): O(log n)  
- Update (point update or range update): O(log n)  
- Memory usage: O(4n)  

**Use cases:**
- Range queries (sum, min, max, gcd).  
- Range updates with **lazy propagation**.  
- Problems like **RMQ (Range Minimum Query)**, **interval sum**, **competitive programming queries**.  

---

## 2. Fenwick Tree (Binary Indexed Tree, BIT)

- A **compact array-based structure** that supports prefix queries using binary decomposition.  
- Each index covers a segment of length equal to the **lowest set bit** of that index.  

**Operations:**
- Prefix sum query: O(log n)  
- Range query [l, r]: prefix(r) - prefix(l-1)  
- Update: O(log n)  
- Memory usage: O(n)  

**Use cases:**
- Fast **prefix/range sum queries**.  
- Situations with frequent updates + queries.  
- Simpler and more space-efficient than segment trees.  

---

## 3. Comparison

| Feature            | Segment Tree | Fenwick Tree (BIT) |
|--------------------|--------------|---------------------|
| Query time         | O(log n)     | O(log n)            |
| Update time        | O(log n)     | O(log n)            |
| Memory             | O(4n)        | O(n)                |
| Complexity         | More complex | Simple              |
| Supports           | Sum, min, max, gcd, range updates (lazy) | Mostly prefix sums (extensions possible) |
| Use cases          | Advanced range queries, competitive problems | Prefix sums, frequency tables, simpler problems |

---

✅ **Rule of thumb:**
- If you only need **range sum queries and point updates** → **Fenwick Tree (BIT)** (simpler, faster in practice).  
- If you need **min/max, gcd, or range updates** → **Segment Tree** (more powerful).  

In [ ]:
use std::ops::{Add, Sub};

// SegmentTree represents a generic segment tree
#[derive(Debug, Clone)]
pub struct SegmentTree<T> {
    n: usize,
    size: usize,
    tree: Vec<T>,
    identity: T,
}

impl<T> SegmentTree<T>
where
    T: Clone + Add<Output = T> + Default,
{
    // Create a new segment tree for sum operations
    pub fn new_sum(data: &[T]) -> Self {
        let n = data.len();
        if n == 0 {
            return SegmentTree {
                n: 0,
                size: 1,
                tree: vec![T::default()],
                identity: T::default(),
            };
        }

        // Find next power of 2
        let mut size = 1;
        while size < n {
            size <<= 1;
        }

        let mut tree = vec![T::default(); 2 * size];
        let identity = T::default();

        // Initialize leaves
        for (i, value) in data.iter().enumerate() {
            tree[size + i] = value.clone();
        }

        // Build internal nodes
        for i in (1..size).rev() {
            tree[i] = tree[2 * i].clone() + tree[2 * i + 1].clone();
        }

        SegmentTree {
            n,
            size,
            tree,
            identity,
        }
    }

    // Get the size
    pub fn len(&self) -> usize {
        self.n
    }

    // Check if empty
    pub fn is_empty(&self) -> bool {
        self.n == 0
    }

    // Get value at index i
    pub fn get(&self, i: usize) -> T {
        if i >= self.n {
            panic!("Index out of bounds");
        }
        self.tree[self.size + i].clone()
    }

    // Set value at index i
    pub fn set(&mut self, i: usize, value: T) {
        if i >= self.n {
            panic!("Index out of bounds");
        }

        let mut idx = self.size + i;
        self.tree[idx] = value;
        idx /= 2;

        while idx >= 1 {
            self.tree[idx] = self.tree[2 * idx].clone() + self.tree[2 * idx + 1].clone();
            idx /= 2;
        }
    }

    // Query range [l, r] inclusive
    pub fn query(&self, l: usize, r: usize) -> T {
        if self.n == 0 {
            panic!("Cannot query empty segment tree");
        }
        if l > r {
            return self.identity.clone();
        }
        if r >= self.n {
            panic!("Range out of bounds");
        }

        let mut res_left = self.identity.clone();
        let mut res_right = self.identity.clone();
        let mut left = l + self.size;
        let mut right = r + self.size;

        while left <= right {
            if left % 2 == 1 {
                res_left = res_left + self.tree[left].clone();
                left += 1;
            }
            if right % 2 == 0 {
                res_right = self.tree[right].clone() + res_right;
                right -= 1;
            }
            left /= 2;
            right /= 2;
        }

        res_left + res_right
    }
}

// Min Segment Tree
impl<T> SegmentTree<T>
where
    T: Clone + Ord + Default,
{
    // Create a new segment tree for min operations
    pub fn new_min(data: &[T], max_value: T) -> Self {
        let n = data.len();
        if n == 0 {
            return SegmentTree {
                n: 0,
                size: 1,
                tree: vec![max_value.clone()],
                identity: max_value,
            };
        }

        // Find next power of 2
        let mut size = 1;
        while size < n {
            size <<= 1;
        }

        let mut tree = vec![max_value.clone(); 2 * size];

        // Initialize leaves
        for (i, value) in data.iter().enumerate() {
            tree[size + i] = value.clone();
        }

        // Build internal nodes
        for i in (1..size).rev() {
            tree[i] = if tree[2 * i] <= tree[2 * i + 1] {
                tree[2 * i].clone()
            } else {
                tree[2 * i + 1].clone()
            };
        }

        SegmentTree {
            n,
            size,
            tree,
            identity: max_value,
        }
    }

    // Query min in range [l, r] inclusive
    pub fn query_min(&self, l: usize, r: usize) -> T {
        if self.n == 0 {
            panic!("Cannot query empty segment tree");
        }
        if l > r {
            return self.identity.clone();
        }
        if r >= self.n {
            panic!("Range out of bounds");
        }

        let mut res_left = self.identity.clone();
        let mut res_right = self.identity.clone();
        let mut left = l + self.size;
        let mut right = r + self.size;

        while left <= right {
            if left % 2 == 1 {
                res_left = if res_left <= self.tree[left] {
                    res_left
                } else {
                    self.tree[left].clone()
                };
                left += 1;
            }
            if right % 2 == 0 {
                res_right = if self.tree[right] <= res_right {
                    self.tree[right].clone()
                } else {
                    res_right
                };
                right -= 1;
            }
            left /= 2;
            right /= 2;
        }

        if res_left <= res_right {
            res_left
        } else {
            res_right
        }
    }

    // Set value for min tree
    pub fn set_min(&mut self, i: usize, value: T) {
        if i >= self.n {
            panic!("Index out of bounds");
        }

        let mut idx = self.size + i;
        self.tree[idx] = value;
        idx /= 2;

        while idx >= 1 {
            self.tree[idx] = if self.tree[2 * idx] <= self.tree[2 * idx + 1] {
                self.tree[2 * idx].clone()
            } else {
                self.tree[2 * idx + 1].clone()
            };
            idx /= 2;
        }
    }
}

// FenwickTree (Binary Indexed Tree)
#[derive(Debug, Clone)]
pub struct FenwickTree<T> {
    n: usize,
    bit: Vec<T>,
    identity: T,
}

impl<T> FenwickTree<T>
where
    T: Clone + Add<Output = T> + Sub<Output = T> + Default,
{
    // Create a new Fenwick tree
    pub fn new(data: &[T]) -> Self {
        let n = data.len();
        let mut ft = FenwickTree {
            n,
            bit: vec![T::default(); n + 1],
            identity: T::default(),
        };

        // Add all initial values
        for (i, value) in data.iter().enumerate() {
            ft.add(i, value.clone());
        }

        ft
    }

    // Get the size
    pub fn len(&self) -> usize {
        self.n
    }

    // Check if empty
    pub fn is_empty(&self) -> bool {
        self.n == 0
    }

    // Point add: arr[i] += delta
    pub fn add(&mut self, i: usize, delta: T) {
        if i >= self.n {
            panic!("Index out of bounds");
        }

        let mut idx = i + 1;
        while idx <= self.n {
            self.bit[idx] = self.bit[idx].clone() + delta.clone();
            idx += idx & (!idx + 1); // idx += idx & -idx in two's complement
        }
    }

    // Prefix sum [0, r] inclusive
    pub fn prefix(&self, r: usize) -> T {
        if r >= self.n {
            return self.prefix(self.n - 1);
        }

        let mut res = self.identity.clone();
        let mut idx = r + 1;
        while idx > 0 {
            res = res + self.bit[idx].clone();
            idx -= idx & (!idx + 1); // idx -= idx & -idx
        }
        res
    }

    // Range query [l, r] inclusive
    pub fn range_query(&self, l: usize, r: usize) -> T {
        if l > r {
            return self.identity.clone();
        }
        if r >= self.n {
            panic!("Range out of bounds");
        }

        if l == 0 {
            self.prefix(r)
        } else {
            self.prefix(r) - self.prefix(l - 1)
        }
    }
}

## Examples

Below are examples showing segment tree and Fenwick tree usage.

In [ ]:
// Segment Tree examples
let arr = vec![2, 1, 3, 4, 5, 2, 7];

// Range Sum Segment Tree
let mut sum_seg = SegmentTree::new_sum(&arr);
println!("sum [0, 6] = {}", sum_seg.query(0, 6)); // 24
println!("sum [2, 4] = {}", sum_seg.query(2, 4)); // 12
sum_seg.set(3, 10); // arr[3] = 10
println!("after update arr[3]=10, sum [2, 4] = {}", sum_seg.query(2, 4)); // 18

// Range Min Segment Tree  
let arr2 = vec![5, 3, 6, 2, 8, 7];
let mut min_seg = SegmentTree::new_min(&arr2, i32::MAX);
println!("min [0, 5] = {}", min_seg.query_min(0, 5)); // 2
println!("min [1, 3] = {}", min_seg.query_min(1, 3)); // 2
min_seg.set_min(3, 9);
println!("after update arr2[3]=9, min [1, 3] = {}", min_seg.query_min(1, 3)); // 3

In [ ]:
// Fenwick Tree examples
let arr = vec![2, 1, 3, 4, 5, 2, 7];
let mut bit = FenwickTree::new(&arr);

println!("prefix(6) = {}", bit.prefix(6)); // 24
println!("range_query(2, 4) = {}", bit.range_query(2, 4)); // 12
bit.add(3, 6); // arr[3] += 6 -> 10
println!("after add(3,6), range_query(2, 4) = {}", bit.range_query(2, 4)); // 18